In [ ]:
import numpy as np
import json
from scipy import sparse
import os

In [ ]:
dataset = 'toy10'
dim = 100
random_seed = 9999744
config = f'data_{dataset}-dim_{dim}-rand_{random_seed}'
save_dir = f'../result/{config}'

In [ ]:
term_doc_sparse = sparse.load_npz(os.path.join(save_dir, f'termDocSparse.npz'))
term_array = np.load(os.path.join(save_dir, f'termArray.npy'))
doc_array  = np.load(os.path.join(save_dir, f'docArray.npy'))
with open(os.path.join(save_dir, 'term-doc.json'), mode='r', encoding='utf8') as f:
    term_doc_text = json.load(f)
    terms = term_doc_text['terms']
    docs = term_doc_text['docs']
with open(os.path.join(save_dir, 'frobenius-norm-approx-raw.txt'), mode='r', encoding='utf8') as f:
    frob_norm = float(f.read())


In [ ]:
print('term_doc_sparse.shape:', term_doc_sparse.shape)
print('term_array.shape', term_array.shape)
print('doc_array.shape', doc_array.shape)
print('#term_text:', len(terms))
print('#doc_text:', len(docs))
print('feature_dim:', doc_array.shape[1])
print('frobenius norm:', frob_norm)